<a href="https://colab.research.google.com/github/apoorvapu/data_science/blob/main/NLP_languageTranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**NLP**: **(2 methods: (1) fine-tuning LLM and (2) model training from scratch)**

# language translation

For fine-tuning Meta-llama2 for language translation with low memory,you can use techniques like LoRA, QLoRA, 8-bit/4-bit quantization, and Retrieval-Augmented Generation (RAG).

In [4]:
#!pip install datasets

In [5]:
#!pip install huggingface_hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/usr/local/bin/huggingface-c

In [6]:
from transformers import LlamaForCausalLM, LlamaTokenizer, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
import torch
from datasets import load_dataset

ModuleNotFoundError: No module named 'datasets'

In [ ]:
# Load dataset
dataset = load_dataset("rahular/itihasa", "en-fr")  # Example: English-French translation
dataset.head()

In [ ]:


# Load tokenizer and model
model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = LlamaTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

# Apply LoRA for memory efficiency
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16, lora_alpha=32, lora_dropout=0.1
)
model = get_peft_model(model, lora_config)


def preprocess_function(examples):
    inputs = ["Translate to French: " + ex for ex in examples["source"]]
    targets = examples["target"]
    model_inputs = tokenizer(inputs, text_target=targets, padding="max_length", truncation=True)
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Training Arguments
training_args = TrainingArguments(
    output_dir="./llama-finetuned",
    per_device_train_batch_size=2,  # Reduce batch size for low memory
    gradient_accumulation_steps=8,  # Helps with small batch sizes
    learning_rate=5e-5,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_steps=10,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
)

# Train model
trainer.train()
